In [1]:
%run ../utils/init_env.py

In [2]:
import sqlite3
import json
import os
import pandas as pd
import nltk

In [8]:
DB_PATH = "/Users/atissera/Developer/repos/unrc-cs-thesis/spider_data/test_database"
GOLD_SQL = "/Users/atissera/Developer/repos/unrc-cs-thesis/data/gold_queries_benchmark.sql"
PRED_SQL = "/Users/atissera/Developer/repos/unrc-cs-thesis/data/predicted_queries_benchmark.sql"

In [16]:
with open(GOLD_SQL) as f:
    glist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]

with open(PRED_SQL) as f:
    plist = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]

In [19]:
plist

[['SELECT s.student_id, s.bio_data, s.student_details FROM Students s JOIN (SELECT b.student_id, COUNT(*) AS COUNT, b.behaviour_monitoring_details FROM Behaviour_Monitoring b GROUP BY b.student_id, b.behaviour_monitoring_details) AS m ON s.student_id = m.student_id WHERE m.COUNT = (SELECT MAX(COUNT) FROM (SELECT b.student_id, COUNT(*) AS COUNT, b.behaviour_monitoring_details FROM Behaviour_Monitoring b GROUP BY b.student_id, b.behaviour_monitoring_details)) ORDER BY s.student_id NULLS LAST']]

In [20]:
import os
import sqlite3
import pandas as pd

for pred, gold in zip(plist, glist):
    db_id = gold[-1]
    pred_query = pred[0]
    gold_query = gold[0]
    db_file = os.path.join(DB_PATH, db_id, db_id + ".sqlite")

    if not os.path.exists(db_file):
        print(f"❌ Database file not found: {db_file}")
        continue

    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Run predicted query
        cursor.execute(pred_query)
        pred_columns = [desc[0] for desc in cursor.description]
        pred_rows = cursor.fetchall()
        pred_df = pd.DataFrame(pred_rows, columns=pred_columns)

        # Run gold query
        cursor.execute(gold_query)
        gold_columns = [desc[0] for desc in cursor.description]
        gold_rows = cursor.fetchall()
        gold_df = pd.DataFrame(gold_rows, columns=gold_columns)

        conn.close()

        # Sort and reset index for fair comparison
        pred_df_sorted = pred_df.sort_values(by=pred_columns, axis=0).reset_index(drop=True)
        gold_df_sorted = gold_df.sort_values(by=gold_columns, axis=0).reset_index(drop=True)

        # Compare
        is_equal = pred_df_sorted.equals(gold_df_sorted)

        print(f"\n📌 Database: {db_id}")
        print("🔹 Predicted Query:")
        print(pred_query)
        print("🔸 Gold Query:")
        print(gold_query)

        print("\n📊 Predicted Result:")
        display(pred_df)
        print("📊 Gold Result:")
        display(gold_df)

        if is_equal:
            print("✅ Predicted and Gold results match.")
        else:
            print("❌ Results do not match.")

    except Exception as e:
        print(f"❌ Error running queries on {db_id}")
        print(e)


📌 Database: cre_Students_Information_Systems
🔹 Predicted Query:
SELECT s.student_id, s.bio_data, s.student_details FROM Students s JOIN (SELECT b.student_id, COUNT(*) AS COUNT, b.behaviour_monitoring_details FROM Behaviour_Monitoring b GROUP BY b.student_id, b.behaviour_monitoring_details) AS m ON s.student_id = m.student_id WHERE m.COUNT = (SELECT MAX(COUNT) FROM (SELECT b.student_id, COUNT(*) AS COUNT, b.behaviour_monitoring_details FROM Behaviour_Monitoring b GROUP BY b.student_id, b.behaviour_monitoring_details)) ORDER BY s.student_id NULLS LAST
🔸 Gold Query:
select t1.bio_data from students as t1 join behaviour_monitoring as t2 on t1.student_id  =  t2.student_id where t2.behaviour_monitoring_details in ( select behaviour_monitoring_details from behaviour_monitoring group by behaviour_monitoring_details order by count(*) desc limit 1 ) except select t1.bio_data from students as t1 join behaviour_monitoring as t2 on t1.student_id  =  t2.student_id where t2.behaviour_monitoring_deta

,student_id,bio_data,student_details
0,435,Vanessa,Suite 684
1,648,Leonard,Suite 505


📊 Gold Result:


,bio_data
0,Felipa
1,Pansy
2,Vanessa


❌ Results do not match.
